In [16]:
!wget https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/9sxpkmm8xn-1.zip

--2023-02-06 00:46:42--  https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/9sxpkmm8xn-1.zip
Resolving prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)... 52.218.91.48, 52.218.62.200, 52.92.34.74, ...
Connecting to prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)|52.218.91.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3325616680 (3.1G) [application/zip]
Saving to: ‘9sxpkmm8xn-1.zip.1’

9sxpkmm8xn-1.zip.1  100%[===================>]   3.10G  32.8MB/s    in 99s     

2023-02-06 00:48:22 (32.0 MB/s) - ‘9sxpkmm8xn-1.zip.1’ saved [3325616680/3325616680]



In [17]:
!7za x /content/9sxpkmm8xn-1.zip


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 3325616680 bytes (3172 MiB)

Extracting archive: /content/9sxpkmm8xn-1.zip
--
Path = /content/9sxpkmm8xn-1.zip
Type = zip
Physical Size = 3325616680

  0%    
Would you like to replace the existing file:
  Path:     ./A Curated Hate Speech Dataset/HSData/README.rtf
  Size:     10255 bytes (11 KiB)
  Modified: 2022-10-03 07:30:22
with the file from archive:
  Path:     A Curated Hate Speech Dataset/HSData/README.rtf
  Size:     10255 bytes (11 KiB)
  Modified: 2022-10-03 07:30:22
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? A

  0% - A Curated Hate Speech Dataset/HSData/README.rtf         

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import string
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU
import tensorflow as tf

In [19]:
df=pd.read_csv('/content/A Curated Hate Speech Dataset/HSData/0_RawData/data_huang_devansh.csv')

In [20]:
df.head()

,Content,Label
0,`- This is not ``creative``. Those are the di...,0
1,` :: the term ``standard model`` is itself le...,0
2,"True or false, the situation as of March 200...",0
3,"Next, maybe you could work on being less cond...",0
4,This page will need disambiguation.,0


In [21]:
df['Label'].unique()

array([0, 1])

In [22]:
df.shape

(842335, 2)

In [23]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [25]:
import re
df['Content'] = df['Content'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [26]:
df

,Content,Label
0,`- This is not ``creative``. Those are the di...,0
1,` :: the term ``standard model`` is itself le...,0
2,"True or false, the situation as of March 200...",0
3,"Next, maybe you could work on being less cond...",0
4,This page will need disambiguation.,0
...,...,...
842330,"""Never really gave it much thought. I just fig...",0
842331,"""Nadie se salva de la regla 34 xd""",0
842332,"""Question: Are you a boy or a girl?""",0
842333,"""Leave your email or phone number and maybe yo...",1


In [27]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)

# Remove Accented Chars
def remove_accented_chars_func(text):
    '''
    Removes all accented characters from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without accented characters
    '''
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
# Remove punctuation
def remove_punctuation_func(text):
    '''
    Removes all punctuation from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without punctuations
    '''
    return re.sub(r'[^a-zA-Z0-9]', ' ', text)
# Remove Irrelevant Char
def remove_irr_char_func(text):
    '''
    Removes all irrelevant characters (numbers and punctuation) from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without irrelevant characters
    '''
    return re.sub(r'[^a-zA-Z]', ' ', text)
#Remove Whitespaces
def remove_extra_whitespaces_func(text):
    '''
    Removes extra whitespaces from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without extra whitespaces
    ''' 
    return re.sub(r'^\s*|\s\s*', ' ', text).strip()
#lemmatization
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    text=remove_accented_chars_func(text)
    text=remove_punctuation_func(text)
    text=remove_irr_char_func(text)
    text=remove_extra_whitespaces_func(text)
    return text
#Apply function on review column
df['Content']=df['Content'].apply(denoise_text)

df

/usr/local/lib/python3.8/dist-packages/bs4/__init__.py:332: UserWarning: "http://t.co/3hqI0L6cwU" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/bs4/__init__.py:332: UserWarning: "http://t.co/bN7be6njVT" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/bs4/__init__.py:332: UserWarning: "http://t.co/BW9r97i4p0" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/bs4/__init__.py:332: UserWarning: "http://t.co/ggVGrdqO

,Content,Label
0,creative dictionary definitions terms insuranc...,0
1,term standard model less NPOV think we d prefe...,0
2,True false situation March such Saudi proposal...,0
3,Next maybe could work less condescending sugge...,0
4,page need disambiguation,0
...,...,...
842330,Never really gave much thought figured back door,0
842331,Nadie se salva de la regla xd,0
842332,Question boy girl,0
842333,Leave email phone number maybe twits meet beat...,1


In [28]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [29]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
  words = text.split()
  words = [lemmatizer.lemmatize(word,pos='v') for word in words]
  return ' '.join(words)
df['Content'] = df['Content'].apply(lemmatize_words)

df

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Content,Label
0,creative dictionary definitions term insurance...,0
1,term standard model less NPOV think we d prefe...,0
2,True false situation March such Saudi proposal...,0
3,Next maybe could work less condescend suggesti...,0
4,page need disambiguation,0
...,...,...
842330,Never really give much think figure back door,0
842331,Nadie se salva de la regla xd,0
842332,Question boy girl,0
842333,Leave email phone number maybe twit meet beat ...,1


In [30]:
dfclean=df.copy()

In [31]:
dfclean.to_csv('cleanmandalay.csv')